In [1]:
%matplotlib notebook
import numpy as np
from rlvs.molecule_world.helper_functions import *
from rlvs.molecule_world.complex import Complex
from rlvs.molecule_world.molecule import Molecule
from rlvs.molecule_world.featurizer import Featurizer

### Load molecules 

In [7]:
print("--- Ligands ----\n")

# load ligand
ligand1 = OB_to_mol(read_to_OB(filename="./data/pdbqt_data/lopinavir.pdbqt", filetype="pdbqt"), mol_type=1)
ligand1.summary()
print("----------\n")
ligand2 = OB_to_mol(read_to_OB(filename="./data/pdbqt_data/ritonavir.pdbqt", filetype="pdbqt"), mol_type=1)
ligand2.summary()


print("\n\n--- Receptor ----\n")
# load receptor
protein = OB_to_mol(read_to_OB(filename="./data/pdbqt_data/6Y2F_MOD.pdbqt", filetype="pdbqt"), mol_type=-1)
protein.summary()

--- Ligands ----

Number of heavy atoms in molecule = 46
Number of features = 15
Max distance between atoms along
x:(-8.08, 7.692, 15.772)
y:(-3.83, 4.011, 7.841)
z:(-3.817, 3.489, 7.306)
----------

Number of heavy atoms in molecule = 49
Number of features = 15
Max distance between atoms along
x:(-8.774, 9.387, 18.161)
y:(-4.406, 5.463, 9.869)
z:(-3.23, 3.093, 6.323)


--- Receptor ----

Number of heavy atoms in molecule = 2341
Number of features = 15
Max distance between atoms along
x:(-29.056, 21.003, 50.059)
y:(-33.198, 32.099, 65.297)
z:(-13.814, 40.634, 54.448)


In [4]:
f = Featurizer()
obmol = read_to_OB(filename="./data/pdbqt_data/ritonavir.pdbqt", filetype="pdbqt")
adg_mat = np.zeros((obmol.NumAtoms(), obmol.NumAtoms()))
for bond in ob.OBMolBondIter(obmol):
    adg_mat[bond.GetBeginAtom().GetIndex(), bond.GetEndAtom().GetIndex()] = 1

ff = np.array([f.get_atom_features(atom, 1) for atom in ob.OBMolAtomIter(obmol) if atom.GetAtomicNum() > 1])

coords, feats, adj_mat = f.get_mol_features(obmol=obmol, molecule_type=1, bond_verbose=0)
print(list(ff[:, 0]))
print(list(coords) == list(ff[:, 0]))


[[4.646, -1.202, -0.383], [4.503, -1.821, -1.419], [3.682, -1.233, 0.559], [5.877, -0.406, -0.159], [6.016, 0.261, 0.97], [7.105, 0.989, 1.214], [7.246, 1.7, 2.422], [8.144, 1.057, 0.25], [8.368, 2.435, 2.645], [9.387, 2.499, 1.696], [9.288, 1.826, 0.516], [8.005, 0.348, -0.958], [6.867, -0.381, -1.152], [2.426, -1.939, 0.298], [1.376, -0.952, -0.142], [1.651, 0.226, -0.23], [0.132, -1.378, -0.439], [-0.888, -0.418, -0.867], [-2.277, -0.974, -0.547], [-2.512, -2.148, -1.326], [-3.337, 0.079, -0.879], [-4.65, -0.383, -0.41], [-5.599, 0.734, -0.319], [-6.909, 0.245, 0.304], [-7.488, -0.88, -0.557], [-8.774, -1.406, 0.078], [-6.461, -2.011, -0.663], [-8.471, -1.956, 1.473], [-7.455, -3.095, 1.359], [-6.164, -2.563, 0.733], [-5.168, -1.456, -1.268], [-5.011, 1.824, 0.538], [-4.11, 1.57, 1.309], [-5.486, 3.082, 0.449], [-4.915, 4.142, 1.283], [-3.424, 4.288, 0.972], [-5.628, 5.463, 0.989], [-5.095, 3.783, 2.759], [-0.768, -0.184, -2.374], [0.553, 0.477, -2.676], [1.668, -0.296, -2.94], [2.8

/Users/in-justin.jose/.miniconda/envs/rl-virtual-screening/lib/python3.6/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


In [ ]:
max_dist = 10
resolution = 0.1

grid = Complex(max_dist=max_dist, resolution=resolution, num_features=ligand1.num_features)
print(grid.box_size, grid.resolution, grid.max_dist)

### Create a box smaller than full protein -> atoms outside will be discarded. Original protein object can be modified with "update_mols" parameter

In [ ]:
grid.update_tensor(mols=[ligand1, protein], update_mols=False)
grid.plot_tensor(feature_axis=-1)

### Manually crop protein and update tensor (now all atoms are within box bounds)

In [ ]:
protein.crop(ligand1.get_centroid(), 5, 5, 5)
grid.update_tensor(mols=[ligand1, protein], update_mols=False)
grid.plot_tensor(feature_axis=-1, protein_alpha=0.3)

### Create bigger box to accomodate full protein

In [ ]:
max_dist = 41
resolution = 1

protein = OB_to_mol(read_to_OB(filename="../../pdbqt_data/6Y2F_MOD.pdbqt", filetype="pdbqt"), mol_type=-1)

grid = Complex(max_dist=max_dist, resolution=resolution, num_features=ligand1.num_features)
print(grid.box_size, grid.resolution, grid.max_dist)
grid.update_tensor(mols=[ligand1, protein], update_mols=False)
grid.plot_tensor(feature_axis=-1)